<a href="https://colab.research.google.com/github/jwang1/classicComputerScienceProblems/blob/master/Trivial_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>